In [2]:
import random
from time import sleep
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.chrome.options import Options
import requests
from bs4 import BeautifulSoup
from termcolor import colored
from timeit import timeit
import threading
import json
import pprint
from datetime import datetime
import time


def current_date(date):
    months = ("Enero", "Febrero", "Marzo", "Abri", "Mayo", "Junio", "Julio", "Agosto", "Septiembre", "Octubre", "Noviembre", "Diciembre")
    day = date.day
    month = months[date.month - 1]
    year = date.year
    messsage = "{} de {} del {}".format(day, month, year)

    return messsage

now = datetime.now()


#driver = webdriver.Chrome("./chromedriver.exe")
#driver.get("https://www.critica.com.pa/mundo")
gizmodo = {} #DICCIONARIO
gizmodo['Articles'] = []
europa_press = {}
europa_press['Articles'] = []
vanguardia = {} #DICCIONARIO
vanguardia['Articles'] = []
todayMund = {}
todayMund['Articles'] = []

URL_ALMIN = 'https://www.lavanguardia.com/alminuto'
URL = 'https://es.gizmodo.com/c/tecnologia'
URL_EUROPA = 'https://www.europapress.es/internacional/'
URL_MUNDOTODAY ='https://www.elmundotoday.com/'
      
def content_page(link):
    link_cont = requests.get(link)
    soup = BeautifulSoup(link_cont.text, 'html.parser')
    container = soup.find_all('p',{'class':'sc-77igqf-0 dQFbxq'})
    parraf = []
    for parrafo in container:
        parrafo
        parraf.append(parrafo.getText())
    return(parraf)
    #return parraf
'''
class Scrapy:
    def __ini__(self,title,author,content,date,urlLink,cont_link,img):
        self.title = title
        self.author = author
        self.content = content
        self.date = date
        self.urlLink = urlLink
        self.cont_link = cont_link
        self.img = img
        
    def __str__(self):
        return str(self.__class__) + ": " + str(self.__dict__)
'''
    


           
def gizmo():
    inicio_giz = time.time()
    re = requests.get(URL)
    codeg = re.status_code
    print(f'Status Gizmodo: ',colored(f'{codeg}','yellow'))
    if re.status_code == 200:
        soup = BeautifulSoup(re.text, 'html.parser')

        if soup is not None:
            articles = soup.find_all('article',{'class':'cw4lnv-0 iTueKC js_post_item'})
            #redires = soup.find_all
            cant_redir = len(soup.find_all('a')) #REDIRECCIONES
            #noticias['Articles'].append({'Redir':cant_redir})
            for article in articles:
                #IMPRIME EL TITULO DEL ARTICULO
                titulo = article.find('h2', {'class':'sc-759qgu-0 iSjOfj cw4lnv-6 fGbMFF'}).getText()
                #IMPRIME EL AUTOR DEL ARTICULO
                autor = article.find('div',{'class':'sc-1mep9y1-0 sc-1ixdk2y-0 fCQpxO sc-1rye1-3 hwANEt'}).text
                #IMPRIME EL LINK DEL ARTICULO
                link = article.find('a',{'class':'sc-1out364-0 hMndXN js_link'}).get('href')
                #IMPRIME EL CONTENIDO DEL ARTICULO
                article_content = content_page(link)
                #IMPRIME LA FECHA DEL ARTICULO
                fecha = article.find('div',{'class':'sc-3nbvzd-1 kpXIm'}).getText()
                #IMPRIME EN LINK DE LA IMAGEN DEL ARTICULO
                if(article.find('img',{'data-format':'jpg'})):
                    imagen = article.find('img',{'data-format':'jpg'}).get('srcset')
                else:
                    imagen = "No Imagen"
                gizmodo['Articles'].append({
                    'Redir':cant_redir,
                    'Title':titulo,
                    'Author':autor,
                    'Link':link,
                    'Content':article_content,
                    'Date':fecha,
                    'Img':imagen
                })

            #print(gizmodo)
            with open('gizmodo.json','w') as f:
                json.dump(gizmodo, f, indent=4)
                               
    print(colored('Archivo de gizmodo listo','green'))
    t = time.time() - inicio_giz
    print(f"Terminado en:",colored(f"{t:.2f}","yellow") +" segundos")
#print("--- %s seconds ---" % (time.time() - inicio_giz))
    
def europaPress():
    inicio_europa = time.time()
    re = requests.get(URL_EUROPA)
    codee = re.status_code
    print(f'Status Europapress: ',colored(f'{codee}','yellow'))
    #print(f'Status Europapress: {codee}')
    if re.status_code == 200:
        soup = BeautifulSoup(re.text, 'html.parser')

        if soup is not None:
            mundo_articles = soup.find_all('article',{'class':'articulo-portada'})
            
            #redires = soup.find_all
            mundo_redir = len(soup.find_all('a')) #REDIRECCIONES
            #print(mundo_redir)
            #noticias['Articles'].append({'Redir':cant_redir})
            for article in mundo_articles:
                mundo_title = article.find('h2',{'class':'articulo-titulo'}).getText()
                #print(mundo_title)
                mundo_fecha = article.find('time',{'class':'hora_hace'}).getText()
                #print(mundo_fecha)
                mundo_link = article.find('a').get('href')
                #print(mundo_link)
                
                link_req = requests.get(mundo_link)
                soup = BeautifulSoup(link_req.text, 'html.parser')
                mundo_container = soup.find_all('div',{'id':'NoticiaPrincipal'})
                for mundo_art in mundo_container:
                    #mundo_autor = mundo_art.find('span',{'class':'fotoPie'})
                    mundo_autor = mundo_art.find('div',{'class':'captionv2'})
                    if(hasattr(mundo_autor,'text')):
                        mundo_autor = mundo_art.find('div',{'class':'captionv2'}).text
                        mundo_autor = mundo_autor.split(" - ")
                        mundo_autor = mundo_autor[1]
                        #print(f'Author: {mundo_autor}')
                    else:
                        mundo_autor = mundo_art.find('div',{'class':'captionv2'})
                        #mundo_autor = mundo_autor.split(" - ")
                        #print(f'Author: {mundo_autor}')
                    mundo_imagen = mundo_art.find('img',{'id':'fotoPrincipalNoticia'})
                    if(hasattr(mundo_imagen,'get')):
                        mundo_imagen = mundo_art.find('img',{'id':'fotoPrincipalNoticia'}).get('src')
                        #print(f'Imagen: {mundo_imagen}')
                    else:
                        mundo_imagen = mundo_art.find('img',{'id':'fotoPrincipalNoticia'})
                        #print(f'Imagen: {mundo_imagen}')
                        
                    mundo_parrafos = soup.find_all('div',{'id':'CuerpoNoticiav2'})
                    mundo_parraf = []
                    for parra in mundo_parrafos:
                        mundo_parraf.append(parra.getText())
                        #print(f'Contenido: {mundo_parraf}')
           
                europa_press['Articles'].append({
                    'Redir':mundo_redir,
                    'Title':mundo_title,
                    'Author':mundo_autor,
                    'Link':mundo_link,
                    'Content':mundo_parraf,
                    'Date':mundo_fecha,
                    'Img':mundo_imagen
                })
                
                #print(europa_press)
                
                with open('europapress.json','w') as f:
                    json.dump(europa_press, f, indent=4)
    print(colored('Archivo de europapress listo','green'))
    eu = time.time() - inicio_europa
    print(f"Terminado en:",colored(f"{eu:.2f}","yellow") +" segundos")

    
def laVanguardia():
    inicio_lavan = time.time()
    re = requests.get(URL_ALMIN)
    codev = re.status_code
    print(f'Status LaVanguardia: ',colored(f'{codev}','yellow'))
    #print(f'Status LaVanguardia: {codev}')
    if re.status_code == 200:
        soup = BeautifulSoup(re.text, 'html.parser')

        if soup is not None:
            van_articles = soup.find_all('li',{'class':'results-item'})
            
            #redires = soup.find_all
            van_redir = len(soup.find_all('a')) #REDIRECCIONES
            #print(van_redir)
            #noticias['Articles'].append({'Redir':cant_redir})
            for van_article in van_articles:
                van_title = van_article.find('h1',{'class':'story-header-title'}).getText()
                #print(van_title)
                van_link = van_article.find('a',{'class':'story-header-title-link'}).get('href')
                #print(van_link)
                #TOMA EL LINK DEL POST EN MINIATURA PARA HACER UN NUEVO REQUEST, PERO ESTA VEZ DESDE DENTRO DEL POST
                link_vanreq = requests.get(van_link)
                soup = BeautifulSoup(link_vanreq.text, 'html.parser')
                van_container = soup.find_all('div',{'class':'detail__main'})
                for van_art in van_container:
                    #CAPTURA EL AUTOR SI ESTA DISPONIBLE LA ETIQUETA
                    if(van_art.find('figcaption',{'class':'img-figcaption'})):
                        van_autor = van_art.find('figcaption',{'class':'img-figcaption'}).text
                        #print(f'Author: {van_autor}')
                    else:
                        van_autor = 'Sin autor'
                        #print(f'Author: {van_autor}')
                    #CAPTURA LA FECHA
                    van_fecha = van_art.find('time',{'class':'d-signature__time'}).text
                    #print(f'Fecha: {van_fecha}')
                    #sI EXISTE LA ETIQUETA DE IMAGEN LA CAPTURA
                    if(van_art.find('img',{'class':'img-responsive'})):
                        van_imagen = van_art.find('img',{'class':'img-responsive'}).get('src')
                        #print(f'Imagen: {van_imagen}')
                    else:
                        van_imagen = 'Imagen no disponible'
                        #print(f'Imagen: {van_imagen}')
                    #BUSCA EL CONTENEDOR PADRE DE LOS PARRAFOS Y CAPTURA CADA PARRAFO
                    van_parrafos = soup.find_all('div',{'itemprop':'articleBody'})
                    van_parraf = []
                    for van_parra in van_parrafos:
                        van_parraf.append(van_parra.getText())
                        #print(f'Contenido: {van_parraf}')

                    vanguardia['Articles'].append({
                        'Redir':van_redir,
                        'Title':van_title,
                        'Author':van_autor,
                        'Link':van_link,
                        'Content':van_parraf,
                        'Date':van_fecha,
                        'Img':van_imagen
                    })

                    #print(vanguardia)

                    with open('vanguardia.json','w') as f:
                        json.dump(vanguardia, f, indent=4)
                    
    print(colored('Archivo de La vanguardia listo','green'))
    va = time.time() - inicio_lavan
    print(f"Terminado en:",colored(f"{va:.2f}","yellow") +" segundos")

    
    
def mundoToday():
    inicio_today = time.time()
    re = requests.get(URL_MUNDOTODAY)
    codeDay = re.status_code
    print(f'Status ElMundoToday: ',colored(f'{codeDay}','yellow'))
    if re.status_code == 200:
        soup = BeautifulSoup(re.text, 'html.parser')
        if soup is not None:
            today_articles = soup.find_all('div',{'class':'item-details'})
            
            today_redir = len(soup.find_all('a'))
            #print(f"Lnks: {today_redir}")
            for today_article in today_articles:
#                
                today_fecha = current_date(now)
                #print(today_fecha)
                today_link = today_article.find('a').get('href')
                #print(today_link)
                
                
                today_linkreq = requests.get(today_link)
                soup = BeautifulSoup(today_linkreq.text, 'html.parser')
                today_container = soup.find_all('div',{'id':'tdb-autoload-article'})
                for today_art in today_container:
                    today_title = today_art.find('h1',{'class':'tdb-title-text'}).getText()
                    #print(today_title)
                    today_autor = today_art.find('div',{'class':'tdb-author-name-wrap'}).text
                    #print(today_autor)
                    today_imagen = today_art.find('img',{'class':'entry-thumb'}).get('src')
                    #print(today_imagen)
                    today_parrafos = soup.select('div.tdb-block-inner > p')
                    today_parraf = []
                    for today_parra in today_parrafos: 
                        today_parraf.append(today_parra.getText())
                        #print(today_parra.getText())
                   
                        
                    todayMund['Articles'].append({
                        'Redir':today_redir,
                        'Title':today_title,
                        'Author':today_autor,
                        'Link':today_link,
                        'Content':today_parraf,
                        'Date':today_fecha,
                        'Img':today_imagen
                    })
                    
                    with open('MundoToday.json','w') as f:
                        json.dump(todayMund, f, indent=4)

    print(colored('Archivo de ELMundoToday listo','green'))
    to = time.time() - inicio_today
    print(f"Terminado en:",colored(f"{to:.2f}","yellow") +" segundos")
    

if __name__ == '__main__':
    threading.Thread(target=gizmo())
    threading.Thread(target=europaPress())
    threading.Thread(target=laVanguardia())
    threading.Thread(target=mundoToday())

   


ModuleNotFoundError: No module named 'termcolor'

In [3]:
import json
import pprint
# json.loads(string)
pprint.pprint ( json.loads(open("gizmodo.json","r").read() ) )

FileNotFoundError: [Errno 2] No such file or directory: 'gizmodo.json'

In [ ]:
from datetime import datetime
day = datetime.now()
print(day)

In [ ]:
from datetime import datetime

def current_date_format(date):
    months = ("Enero", "Febrero", "Marzo", "Abri", "Mayo", "Junio", "Julio", "Agosto", "Septiembre", "Octubre", "Noviembre", "Diciembre")
    day = date.day
    month = months[date.month - 1]
    year = date.year
    messsage = "{} de {} del {}".format(day, month, year)

    return messsage

now = datetime.now()
print(current_date_format(now))

